In [1]:
import sys, os, itertools
import re, nltk
from nltk import pos_tag
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import sklearn, gensim
from sklearn.decomposition import PCA
from gensim.corpora import Dictionary
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
os.chdir(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/')
from utils import preprocess_for_bow
from models.lda import LDAwrappers
from models.hdp import HDPwrapper
from models.gsdmm import MovieGroupProcessWrapper
from models.lftm import LFTMwrapper
from models.bertopic import BERTopicWrapper
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data = preprocess_for_bow('data.csv', preprocessing=False)

In [3]:
data.keys()

dict_keys(['text', 'ids'])

In [4]:
bertopic=BERTopicWrapper(data['text'], data['ids'], custom_vectorizer=True)

In [5]:
bertopic.model.fit(data['text'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [6]:
len(bertopic.model.topics_) #list of topic indexes
#bertopic.model.topic_representations_
#bertopic.model.probabilities_ #probs assigned topics per documents
#bertopic.model.topic_aspects_
#bertopic.model.approximate_distribution(data['text'])[0].shape

30192

## TOPIC representations

In [7]:
bertopic.model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,12318,-1_covid_trump_vaccine_people,"[covid, trump, vaccine, people, covid19, biden...",[I'm with you out of voting for the two evil c...
1,0,578,0_masks_mask_face_face masks,"[masks, mask, face, face masks, wear, wearing,...",[Wearing face masks creates a pH imbalance in ...
2,1,577,1_coronavirus_italy_patients_hospital,"[coronavirus, italy, patients, hospital, bodie...","[Mass dead bodies of coronavirus victims, Vide..."
3,2,562,2_pope_church_catholic_bishops,"[pope, church, catholic, bishops, cardinal, fr...",[Some four months before Archbishop Viganò’s t...
4,3,550,3_tweet_misleading open_helpful context_hit co...,"[tweet, misleading open, helpful context, hit ...",[If you want to add helpful context to any Twe...
...,...,...,...,...,...
658,657,6,657_religion_public schools_teaching religion_...,"[religion, public schools, teaching religion, ...","[In late 2020, a consumer in Illinois filed a ..."
659,658,6,658_coronavirus hoax_great reset_4thisbut_hits...,"[coronavirus hoax, great reset, 4thisbut, hits...",[Does anyone think it's a coincidence that the...
660,659,6,659_ron howard_howard_howard wrote_director ron,"[ron howard, howard, howard wrote, director ro...",[Lmfao soooooo did those Obama phones really g...
661,660,6,660_china country_humanrights_humanrights hist...,"[china country, humanrights, humanrights histo...",[#China is a country that is always committed ...


## fitting the docs with -1 label

In [8]:
idx_to_keep = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i]==-1]
[data['text'][i] for i in idx_to_keep]

['I can even go so far to say that I think the thing (mood) boys have when they think they over-smarted the media and cured a pandemic is cute and I truly feel like a b*tch to poke on that. I thought about it a lot not say anything to avoid make people angry would be selfish.',
 'Election Fraud Covid19 The V.a.c.c.i.n.e. Satanic Pedophilia Masks Social Distancing New World Order BLM ANTIFA White Privilege Racism Capitol Insurrection Cities On Fire Gun Control Assault Rifle Ban Destroy the US Constitution Slavery Reparations The List Goes On... #TheBigLie',
 'Day 48. No microchips in vaccines. COVID didn’t suddenly disappear on Nov 4 like the GOP predicted. No socialism deadly migrant caravans drive thru abortion clinics ANTIFA MS13. But this new guy is great. He’s done way more than golfing guy did his entire 4 years in office.',
 'Hello can one report a channel that deals essentially in misleading information (e.g. about COVID-19 masks etc.) and in conspiracy theories (Qanon-style stu

### examples for 2_pope_church_catholic_bishops

In [9]:
idx_to_keep = [i for i in range(len(bertopic.model.topics_)) if bertopic.model.topics_[i]==2]
[data['text'][i] for i in idx_to_keep]

['banned from conducting religious duties because of his claims that the Covid-19 outbreak was a “pseudo-pandemic” and that it was being used as an excuse to microchip the public. The priest is also said to have complained that the closure of churches in Russia during the -',
 'A Thread.. 🛐 A Plenary Indulgence is granted on the feast of the Divine Mercy for a person who goes to confession and receives holy communion. However during this time of the pandemic wherein the sacraments are unavailable the following are offered for our spiritual benefit:',
 'I’m working on a small article for reflecting on parenting and spiritual formation during the pandemic but after the questions my 7yo just asked me during bath time about baptism and Christ’s human/divine nature I’m going to have to make some edits.',
 'My people are destroyed for lack of knowledge: because thou hast rejected knowledge I will also reject thee that thou shalt be no priest to me: seeing thou hast forgotten the law of thy G